# Part B Question 3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

batch_size=32
num_epochs=5
learning_rate=1e-3

model = models.resnet50(pretrained=True)

# Freezing all layers except the final layer
for param in model.parameters():
    param.requires_grad = False

# Replacing the final fully connected layer with a new one for 10 classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  # Changing 10 to the number of classes in my dataset

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

vanilla_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Loading the dataset
dataset = ImageFolder('/kaggle/input/dataset/inaturalist_12K/train', transform=train_transform)
train_indices, val_indices = train_test_split(range(len(dataset)), test_size=0.2, shuffle=True, stratify=dataset.targets)
train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

test_dataset = ImageFolder('/kaggle/input/dataset/inaturalist_12K/val', transform=vanilla_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
if torch.cuda.is_available():
    model=model.cuda()
    criterion=criterion.cuda()

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    model.train()  # Setting model to training mode
    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = labels.cuda()

        # Forward pass
        outputs = model(images)
        train_loss = criterion(outputs, labels)

        # Backward and optimizer
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

    print(f'At end of epoch [{epoch+1}/{num_epochs}], training loss: {train_loss.item():.4f}')

    # Validation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.cuda()
            labels = labels.cuda()

            outputs = model(images)
            val_loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100 * correct / total
    print(f'At end of epoch [{epoch+1}/{num_epochs}], validation loss: {val_loss.item():.4f}, validation acuracy: {val_accuracy:.2f}%')


print('Finished Training')
# Test phase
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = 100 * correct / total
print(f'Test accuracy: {test_accuracy:.2f}%')

At end of epoch [1/5], training loss: 1.2658
At end of epoch [1/5], validation loss: 0.9687, validation acuracy: 73.00%
At end of epoch [2/5], training loss: 0.9591
At end of epoch [2/5], validation loss: 1.1460, validation acuracy: 74.05%
At end of epoch [3/5], training loss: 1.1457
At end of epoch [3/5], validation loss: 0.3460, validation acuracy: 72.25%
At end of epoch [4/5], training loss: 0.9737
At end of epoch [4/5], validation loss: 0.7586, validation acuracy: 71.30%
At end of epoch [5/5], training loss: 0.5944
At end of epoch [5/5], validation loss: 1.0563, validation acuracy: 73.65%
Finished Training
Test accuracy: 74.25%
